In [2]:
from pathlib import Path
from pprint import pprint
from ost import Sentinel1Batch
from ost.helpers import vector, raster
import json
import gdal
import os

# load username and pw to access sentinel files
username = json.load(open('login_data.json'))['username']
pw = json.load(open('login_data.json'))['pw']

# define the project directory
project_dir = Path.home() / "Documents" / "Python" / "sen1ost" / "data"

In [12]:
niedersachsen_fields = json.load(open('data/aoi/niedersachsen.geojson'))

# define the start and end date
start = "2021-12-01"
end = "2021-12-31"

# define product
key = "ASCENDING_VVVH"

In [13]:
target_len = 15

for field_no in range(1,5):
    single_field = niedersachsen_fields['features'][field_no]
    output = {"type":"FeatureCollection","features":[]}

    len_list = sorted([[idx, len(elem[0])] for idx, elem in enumerate(single_field['geometry']['coordinates'])],key=lambda x: x[1],reverse=True)[:target_len]
    reg_len = [i[0] for i in len_list]

    # mutlipolygon to polygon
    for idx, poly in enumerate(single_field['geometry']['coordinates']):
        if len(single_field['geometry']['coordinates'])<target_len or idx in reg_len:
            xfeature = {"type":"Feature", "geometry":{"type":"Polygon"}}
            xfeature['geometry']['coordinates'] = poly
            output['features'].append(xfeature)
            open('data/aoi/tmp_shape.geojson', 'w').write(json.dumps(output,separators=(',',':'),ensure_ascii=False).replace('}},','}},\n'))

    # aoi file name
    aoi_file = 'data/aoi/tmp_shape.geojson'

    # create buffer around aoi
    vector.buffer_shape(aoi_file, str(project_dir) + '/aoi/aoi_tmp.geojson' ,buffer=0.01)
    aoi = vector.aoi_to_wkt(str(project_dir) + '/aoi/aoi_tmp.geojson')

    # initialize the class to s1_grd instance
    s1_grd = Sentinel1Batch(
        project_dir=project_dir,
        aoi=aoi,
        start=start,
        end=end,
        product_type="GRD",
        beam_mode="IW",
        polarisation="*",
        ard_type="OST-RTC", #Radiometrically Terrain Corrected, images are corrected for radiometric distortions along mountainous slopes
        max_workers=4,
    )

    # search for files
    # added scihub username and pw in Project.py line 260
    s1_grd.search()

    # refine inventory
    s1_grd.refine_inventory()

    # download ascending orbit images
    s1_grd.download(s1_grd.refined_inventory_dict[key], concurrent=10, mirror = '2', uname=username, pword=pw)

    # set ard parameters
    s1_grd.ard_parameters["single_ARD"]["resolution"] = 10  # set output resolution to 100m
    s1_grd.ard_parameters["single_ARD"]["remove_speckle"] = True  # apply a speckle filter
    s1_grd.ard_parameters["single_ARD"]["to_db"] = True

    # preprocess
    s1_grd.grds_to_ards(
        inventory_df=s1_grd.refined_inventory_dict[key],
        timeseries=False,
        timescan=False,
        mosaic=True,
        overwrite=True,
        max_workers=8,
    )

    # convert to tiff and cut to aoi shape
    for u in ['Gamma0_VV_db','Gamma0_VH_db']:
        paths = list(Path(project_dir).rglob(str(u + '.img')))
        for i in paths:
            inpath = str(i)
            outpath = str(project_dir) + '/out/' + single_field['properties']['FLIK'] + '_' + u + '_' + os.path.split(os.path.split(os.path.split(i)[0])[0])[1] + '.tiff'
            ds = gdal.Open(inpath)
            ds = gdal.Translate(outpath, ds)
            ds = None
            raster.mask_by_shape(outpath,outpath, str(project_dir) + '/aoi/tmp_shape.geojson', rescale=False, description=False, to_db=False)

 INFO (13:07:04): Created project directory at /home/david/Documents/Python/sen1ost/data
 INFO (13:07:04): Downloaded data will be stored in: /home/david/Documents/Python/sen1ost/data/download.
 INFO (13:07:04): Inventory files will be stored in: /home/david/Documents/Python/sen1ost/data/inventory.
 INFO (13:07:04): Processed data will be stored in: /home/david/Documents/Python/sen1ost/data/processing.
 INFO (13:07:04): Using /home/david/Documents/Python/sen1ost/data/temp as directory for temporary files.
 INFO (13:07:05): Writing inventory data to geopackage file: /home/david/Documents/Python/sen1ost/data/inventory/full.inventory.gpkg
 INFO (13:07:05): Coverage analysis for ASCENDING tracks in VV VH polarisation.
 INFO (13:07:05): 9 frames for ASCENDING tracks in VV VH polarisation.
 INFO (13:07:05): 9 frames remain after double entry removal
 INFO (13:07:05): Excluding track 15
 INFO (13:07:05): Excluding track 117
 INFO (13:07:05): 5 frames remain after non-AOI overlap
 INFO (13:07:

/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/search.py:212: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  geo_df = geo_df.append(gdf)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/Project.py:352: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  ) = refine_inventory.search_refinement(
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/refine_inventory.py:246: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_frame = out_frame.append(gdf_date)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the sing

 INFO (13:07:05): Found 5 full coverage mosaics.
 INFO (13:07:05): Coverage analysis for DESCENDING tracks in VV VH polarisation.
 INFO (13:07:05): 10 frames for DESCENDING tracks in VV VH polarisation.
 INFO (13:07:05): 10 frames remain after double entry removal
 INFO (13:07:05): Excluding track 66
 INFO (13:07:05): Excluding track 139
 INFO (13:07:05): 4 frames remain after non-AOI overlap
 INFO (13:07:05): All remaining tracks fully overlap the AOI. Not removing anything.
 INFO (13:07:05):  4 frames remain after removal of non-full AOI crossing


/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/refine_inventory.py:433: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  temp_df = temp_df.append(track_gdf)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-null crs provided).
  warnings.warn(
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/refine_inventory.py:451: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  out_frame = out_frame.append(temp_df)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/geopandas/array.py:1406: UserWarning: CRS not set for some of the concatenation inputs. Setting output's CRS as WGS 84 (the single non-n

 INFO (13:07:06): Found 4 full coverage mosaics.
--------------------------------------------
 Summing up the info about mosaics
--------------------------------------------

 5 mosaics for mosaic key ASCENDING_VVVH

 4 mosaics for mosaic key DESCENDING_VVVH
 INFO (13:07:06): One or more scene(s) need(s) to be downloaded.


...

 INFO: Downloading : 964MB [02:24, 6.66MB/s]                           
 INFO: Downloading : 989MB [02:24, 6.84MB/s]                           


 INFO (13:20:13): S1A_IW_GRDH_1SDV_20211206T170037_20211206T170102_040891_04DAFD_F085.zip passed the zip test.
 INFO (13:20:14): S1B_IW_GRDH_1SDV_20211212T165953_20211212T170018_029995_0394BB_F8BB.zip passed the zip test.
 INFO (13:20:14): All products are downloaded.
 INFO (13:20:14): Deleting processing folder to start from scratch
 INFO (13:20:14): Processing acquisition from 20211218 over track 44.
 INFO (13:20:14): Processing acquisition from 20211212 over track 44.
 INFO (13:20:14): Processing acquisition from 20211206 over track 44.
 INFO (13:20:14): Processing acquisition from 20211230 over track 44.
Executing processing graph
Executing processing graph
Executing processing graph
Executing processing graph
15%30%45%60%.70%15%30%45%60%.70%15%30%45%60%.70%.90%.90%15%30%45%60%.70%.90%.90% done.
 done.
 done.
 done.
Executing operator...
20%60%140%100% done.
Writing...
.13%Executing operator...
20%60%100% done.
.Writing...
.13%60%Executing operator...
20%Executing operator...
20%14

/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/grd_batch.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/grd_batch.py:137: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processing_df = processing_df.append(temp_df)


60%100%180% done.
Writing...
.13%100%100%180% done.
Writing...
.13%180%100% done.
60%Writing...
.13%.60%.60%100% done.
.60%100% done.
100% done.


/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/grd_batch.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/grd_batch.py:137: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  processing_df = processing_df.append(temp_df)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/grd_batch.py:86: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append(temp_df)
/home/david/Documents/Python/sen1ost/.venv/lib/python3.8/site-packages/ost/s1/grd_batch.py:137: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.co